# Bibliotecas

In [1]:
import pandas as pd # Biblioteca necessária para a manipulação de dataframes
import numpy as np
from openpyxl import load_workbook

# EXTRACT

Extração das informações presentes no arquivo NintendoGames.csv

In [2]:
df = pd.read_csv('NintendoGames.csv')

# TRANSFORM

Nesse passo é importante entendermos como o nosso banco de dados se comporta.
As notas podem ser entre 0 e 100 quando feita por críticos especializados do mercado e de 0 a 10 quando feita pelo publíco em geral.

Para fins de termos uma análise mais limpa e fiel vamos remover todos os jogos que não possuem uma avaliação tanto dos críticos como dos publíco, Isso é importante pois o banco de dados possui listado jogos que nem mesmo foram lançados no momento da extração dos dados.

In [3]:
df.dropna(subset = ['meta_score','user_score'], inplace = True)

Com essa limpeza podemos começar a estabelecer os parâmetros das nossas análises:

1- Um arquivo excel será gerado onde cada uma das plataformas presentes no banco de dados serão armazenadas em uma aba do arquivo excel "Geral_Analysis.xlsx" onde seus jogos serão organizados em ordem decrescente de nota, sendo a nota final calculada como a media entre as notas dos usuários e a nota dos críticos. A notas dos críticos será normalizada para ficar entre 0 e 10;

2- Também será criado uma aba chamada "PLATAFORM RANK" que armazenará uma classificação em ordem decrescente entre todas as plataformas presente no banco de dados.

In [4]:
# Obtenção de todas as plataformas presentes 

plataforms = df['platform'].drop_duplicates().to_list()

# Cálculo da nota final

df['final_score'] = ((df['meta_score']/10) + df['user_score'])/2

# Criação do df contendo as informações agregadas por plataforma (Aba "Total")

df_plataforms_score = df[['platform','final_score']].groupby(by = 'platform').describe()
df_plataforms_score = df_plataforms_score['final_score'][['count','mean']].reset_index('platform') #Organizando as colunas multi index
df_plataforms_score.sort_values(by='mean', ascending=False, inplace = True) # Organizando da plataforma com a maior avalização para a menor
df_plataforms_score.rename(columns = {'count':'games_count','mean':'score_mean'}, inplace = True) # Renomeando colunas
                        

# LOAD 

Com esses tratamentos básicos se torna possível efetuar a criação dos diferentes arquivos excel.

Uma coisa interessante de se notar é que ainda antes do carregamento irá existir um tratamento de organização dos valores para a alocação dos mesmos na planilha.

In [5]:
def sheets_creation(df,platform):

    df_platform = df.loc[df['platform'] == platform]
    df_platform.sort_values(by='final_score', ascending=False, inplace = True) # Organizando da plataforma com a maior avalização para a menor

    with pd.ExcelWriter('Geral_Analysis.xlsx', engine='openpyxl', mode='a', if_sheet_exists= 'replace') as writer:
    # Salve o novo dataframe em uma nova aba
        df_platform.to_excel(writer, sheet_name=f'{platform} RANK', index=False)

for platform in plataforms:

    sheets_creation(df,platform)

with pd.ExcelWriter('Geral_Analysis.xlsx', engine='openpyxl', mode='a', if_sheet_exists= 'replace') as writer:
    # Salve o novo dataframe em uma nova aba
        df_plataforms_score.to_excel(writer, sheet_name='PLATAFORM RANK', index=False)

C:\Users\guilh\AppData\Local\Temp\ipykernel_16684\2362188473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_platform.sort_values(by='final_score', ascending=False, inplace = True) # Organizando da plataforma com a maior avalização para a menor
